# 1. Installing Dependencies

In [8]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install tensorflow-hub
!pip install opencv-python
!pip install mediapipe
!pip install sklearn
!pip install matplotlib

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [39 lines of output]
  Traceback (most recent call last):
    File "C:\Users\saika\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
      parsed = _parse_requirement(requirement_string)
    File "C:\Users\saika\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
      return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
    File "C:\Users\saika\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
      url, specifier, marker = _parse_requirement_details(tokenizer)
    File "C:\Users\saika\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 126, in _parse_requirement_details
      

# 2. Importing Libraries

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import os

# 3. Setting Up MediaPipe

In [10]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR TO RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR CONVERSION RGB TO BGR
    return image, results

def draw_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

# 4. Loading Model

In [59]:
from tensorflow.keras.models import load_model
model = load_model('Anomaly.h5')

actions = np.array(["Punch", "Crawl", "Run", "Walk"])

# 5. Some Display Features

In [60]:
from scipy import stats

try:
    os.mkdir('Results')
except:
    pass

colors = [(16,117,245), (117,245,16)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,40+num*30), (int(prob*100), 70+num*30), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 60+num*30), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (255,255,255), 1, cv2.LINE_AA)
        
    return output_frame

# 6. Detection

In [62]:
import random as r

In [67]:
import uuid

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5
video=None

cap = cv2.VideoCapture(os.path.join('test', 'Walk.mp4'))
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
    
        # Read feed
        ret, frame = cap.read()

        if video is None:
            video = cv2.VideoWriter(os.path.join('Results', f"Result_{uuid.uuid1()}.mp4"),  
                         cv2.VideoWriter_fourcc(*'MP4V'), 
                         10, frame.shape[-2::-1] )
    
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
    
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        # keypoints = keypoints.reshape(306)
        # print(keypoints.shape)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
    
            if len(sentence) > 5: 
                sentence = sentence[-5:]
    
            # Viz probabilities
            image = prob_viz([max(res[:-1]), res[-1]], ['Anomaly', 'Normal'], image, colors)
            
            
        cv2.rectangle(image, (0,0), (640, 30), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 20), 
                       cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)


        video.write(image)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
    
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    video.release()

1/1 [==============================] - 0s 31ms/step
Run
1/1 [==============================] - 0s 32ms/step
Run
1/1 [==============================] - 0s 30ms/step
Run
1/1 [==============================] - 0s 33ms/step
Run
1/1 [==============================] - 0s 32ms/step
Run
1/1 [==============================] - 0s 32ms/step
Run
1/1 [==============================] - 0s 33ms/step
Run
1/1 [==============================] - 0s 36ms/step
Run
1/1 [==============================] - 0s 32ms/step
Run
1/1 [==============================] - 0s 32ms/step
Run
1/1 [==============================] - 0s 32ms/step
Run
1/1 [==============================] - 0s 32ms/step
Run
1/1 [==============================] - 0s 37ms/step
Run
1/1 [==============================] - 0s 33ms/step
Run
1/1 [==============================] - 0s 34ms/step
Run
1/1 [==============================] - 0s 32ms/step
Run
1/1 [==============================] - 0s 40ms/step
Run
1/1 [==============================] - 0s 35ms/s

# Fix Bugs

In [19]:
cap.release()
cv2.destroyAllWindows()
# video.release()

In [36]:
str(uuid.uuid1())

'c1cb13f3-733e-11ee-8786-4ae7da6c1889'